# Apply FS in ANN

In [38]:
! pip install pandas
! pip install numpy
! pip install sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []

# Import data & treat it

In [39]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)


# Import libraries:

In [40]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers

# parameters for ANN
splits = 7
epochs = 80

# divide into features and label
X = data.iloc[:, 0:14]
y = data.loc[:, 'Offers']

# Do first prediciton with first feature in list from Linear Regression FS:

In [41]:
# data set to append here (start with first feature added)
X_ = X.loc[:,'PrevDay']

# do first prediction
X_.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X_ = X_.astype('float64')
X_ = X_.round(20)

X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)

X_train = np.array(X_train)
X_train = X_train.reshape(-1, 1)
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 1)

sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 1))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.3))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = splits)    
regressor = regressor_tunning()

# train model
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    regressor.fit(X_train_split, y_train_split,  
                         shuffle = False, 
                         validation_split = 0.2,
                         batch_size = 20, 
                         epochs = epochs)

# make predictions and evaluate for all regions
y_pred = regressor.predict(X_test)

# =============================================================================
# METRICS EVALUATION (1) for the whole test set
# =============================================================================
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

# calculate metrics
rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)

# append to list
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)

# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# download spike indication binary set
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)

# append ot lists
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)

# append to list
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 2s 4ms/step - loss: 13329.3499 - mse: 13329.3496 - mae: 109.9059 - val_loss: 34606.3645 - val_mse: 34606.3633 - val_mae: 132.6837
Epoch 2/80
500/500 [==============================] - 0s 412us/step - loss: 13160.9637 - mse: 13160.9639 - mae: 109.1406 - val_loss: 34267.3100 - val_mse: 34267.3125 - val_mae: 131.4041
Epoch 3/80
500/500 [==============================] - 0s 323us/step - loss: 12643.7500 - mse: 12643.7490 - mae: 106.7763 - val_loss: 33244.0137 - val_mse: 33244.0156 - val_mae: 127.4637
Epoch 4/80
500/500 [==============================] - 0s 421us/step - loss: 11311.3102 - mse: 11311.3096 - mae: 100.2896 - val_loss: 30429.6087 - val_mse: 30429.6055 - val_mae: 115.9401
Epoch 5/80
500/500 [==============================] - 0s 389us/step - loss: 7972.2339 - mse: 7972.2339 - mae: 80.9776 - val_loss: 24349.9543 - val_mse: 24349.9531 - val_mae: 85.9359
Epoch 6/80
500/500 [==========

# Apply loop for the rest of the list of features with condition of improvement:

In [42]:
# features list; order made according to Linear Regression FS
features_list = ['APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

best_score = rmse_error

for i in features_list: 
    
   # X_recovery = X_.copy()
    X_recovery = X_
    
    X_ = pd.concat([X_, X.loc[:,i]], axis = 1)
                    
    X_.fillna(method = 'ffill', inplace = True)
    y.fillna(method = 'ffill', inplace = True)

    X_ = X_.astype('float64')
    X_ = X_.round(20)

    X_train, X_test, y_train, y_test = train_test_split(
         X_, y, test_size = 0.15, shuffle = False)
    
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)

    # possible debug
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = len(X_.columns)))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
        for layer in range(n_hidden):
            model.add(Dense(units = n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.3))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = 0.001)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = splits)    
    regressor = regressor_tunning()

    # train model
    for train_index, test_index in tscv.split(X_train):
        X_train_split, X_test_split = X_train[train_index], X_train[test_index]
        y_train_split, y_test_split = y_train[train_index], y_train[test_index]
        regressor.fit(X_train_split, y_train_split,  
                             shuffle = False, 
                             validation_split = 0.2,
                             batch_size = 20, 
                             epochs = epochs)

    # make predictions and evaluate for all regions
    y_pred = regressor.predict(X_test)

    # =============================================================================
    # METRICS EVALUATION (1) for the whole test set
    # =============================================================================
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    # calculate metrics
    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)

    # append to list
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # METRICS EVALUATION (2) on spike regions
    # =============================================================================

    # download spike indication binary set
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)

    # append ot lists
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # METRIC EVALUATION (3) on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)

    # append to list
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

    # condition of improvement for FS
    if best_score < rmse_gen[-1]:
        X_ = X_recovery
    else:
        X_ = X_
        best_score = rmse_error


Train on 500 samples, validate on 125 samples
Epoch 1/80
500/500 [==============================] - 2s 4ms/step - loss: 13337.7836 - mse: 13337.7832 - mae: 109.9448 - val_loss: 34632.8240 - val_mse: 34632.8203 - val_mae: 132.7837
Epoch 2/80
500/500 [==============================] - 0s 355us/step - loss: 13220.8962 - mse: 13220.8975 - mae: 109.4137 - val_loss: 34403.7675 - val_mse: 34403.7734 - val_mae: 131.9225
Epoch 3/80
500/500 [==============================] - 0s 312us/step - loss: 12880.8811 - mse: 12880.8809 - mae: 107.8659 - val_loss: 33738.0407 - val_mse: 33738.0391 - val_mae: 129.3884
Epoch 4/80
500/500 [==============================] - 0s 362us/step - loss: 12009.0474 - mse: 12009.0459 - mae: 103.7222 - val_loss: 31945.5021 - val_mse: 31945.5020 - val_mae: 122.3053
Epoch 5/80
500/500 [==============================] - 0s 306us/step - loss: 9782.0537 - mse: 9782.0537 - mae: 92.1725 - val_loss: 27696.7352 - val_mse: 27696.7363 - val_mae: 103.6013
Epoch 6/80
500/500 [=========

In [45]:
features_list_index =   ['PrevDay',
                         'APXP', 
                         'LOLP',  
                         'In_gen',
                         'Ren_R',
                         'DA_imb_France', 
                         'Rene',
                         'ratio_offers_vol',
                         'DA_price_france',
                         'TSDF',
                         'dino_bin',
                         'DA_margin',
                         'Im_Pr']

results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,}, index = features_list_index)

results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
PrevDay,30.869098,21.861864,43.221078,32.066380,28.550797,20.325845
APXP,30.299779,21.095644,41.852143,31.067724,28.153355,19.594611
LOLP,30.296505,21.040219,41.849467,30.979879,28.149901,19.544066
In_gen,30.326971,21.207724,41.704116,30.869534,28.220027,19.753394
Ren_R,30.688847,21.450446,41.143979,30.648447,28.788173,20.065931
DA_imb_France,30.420877,21.479184,41.842518,31.174677,28.305347,20.019783
Rene,30.525093,21.410706,41.433856,30.701346,28.524075,20.012246
ratio_offers_vol,31.158010,21.661340,42.488293,30.328342,29.072662,20.356752
DA_price_france,30.292605,20.753558,41.790493,30.461410,28.158258,19.292297
TSDF,29.968841,20.369317,41.244563,30.062999,27.879448,18.910190


In [44]:
X_.columns

Index(['PrevDay', 'APXP', 'LOLP', 'DA_price_france', 'TSDF', 'Im_Pr'], dtype='object')

In [46]:
best_score

29.736896274923556